<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/MaestroMostAll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  git+https://github.com/rongardF/tvdatafeed tradingview-screener
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Exponential Moving Average
def ema(series, length):
    return series.ewm(span=length, adjust=False).mean()

# MOST Calculation
def MOST(data, percent, n1):
    df = data.copy()
    percent = percent / 100
    df['EMA'] = ema(df['Close'], length=n1)
    df['tempema'] = 0.0
    df['trend'] = -1
    df['MOST'] = 0.0
    df = df.dropna().reset_index()

    for i in range(1, len(df)):
        if df['trend'][i - 1] == 1:
            df.loc[i, 'tempema'] = max(df['tempema'][i - 1], df['EMA'][i])
        elif df['trend'][i - 1] == -1:
            df.loc[i, 'tempema'] = min(df['tempema'][i - 1], df['EMA'][i])

        if df['EMA'][i] >= df['MOST'][i - 1] and df['trend'][i - 1] == 1:
            df.loc[i, 'trend'] = 1
            df.loc[i, 'MOST'] = df['tempema'][i] * (1 - percent)
        elif df['EMA'][i] <= df['MOST'][i - 1] and df['trend'][i - 1] == -1:
            df.loc[i, 'trend'] = -1
            df.loc[i, 'MOST'] = df['tempema'][i] * (1 + percent)
        elif df['EMA'][i] >= df['MOST'][i - 1] and df['trend'][i - 1] == -1:
            df.loc[i, 'trend'] = 1
            df.loc[i, 'MOST'] = df['tempema'][i] * (1 - percent)
        elif df['EMA'][i] <= df['MOST'][i - 1] and df['trend'][i - 1] == 1:
            df.loc[i, 'trend'] = -1
            df.loc[i, 'MOST'] = df['tempema'][i] * (1 + percent)

    df['Entry'] = df['trend'] == 1
    return df

# Zaman dilimi ismini döndürme fonksiyonu
def interval_name(interval):
    interval_map = {
        Interval.in_15_minute: '15 Dakika',
        Interval.in_30_minute: '30 Dakika',
        Interval.in_45_minute: '45 Dakika',
        Interval.in_1_hour: '1 Saat',
        Interval.in_2_hour: '2 Saat',
        Interval.in_4_hour: '4 Saat',
        Interval.in_daily: 'Günlük',
        Interval.in_weekly: 'Haftalık',
        Interval.in_monthly: 'Aylık'
    }
    return interval_map.get(interval, 'Günlük')

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }
    print("Maestro Most Tarama // Period Girişi seçenekleri:")
    for key, (name, _) in intervals.items():
        print(f"{key}: {name}")

    choice = input("Zaman dilimini girişi (1-9) arası rakam : ")
    interval = intervals.get(choice, ('Günlük', Interval.in_daily))[1]

    return interval

# Main function
def main():
    tv = TvDatafeed()
    interval = get_interval_choice()

    # Get stocks from the Turkey market
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)

    # Prepare the DataFrame to hold results
    Titles = ['Hisse Adı', 'Son Fiyat', 'Giriş Sinyali']
    df_signals = pd.DataFrame(columns=Titles)

    for hisse in Hisseler:
        try:
            # Fetch historical data for the selected interval
            data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval, n_bars=500)

            if not isinstance(data, pd.DataFrame):
                continue  # Skip if the data is not a DataFrame

            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()

            # Calculate MOST
            MostTrend = MOST(data, percent=2, n1=14)  # Change parameters as needed
            Signals = MostTrend.tail(2)

            Buy = Signals['Entry'].iloc[-1]  # Latest entry signal
            Last_Price = Signals['Close'].iloc[-1]

            # Only store if there is a buy signal
            if Buy:
                L1 = [hisse, Last_Price, str(Buy)]
                df_signals.loc[len(df_signals)] = L1
                print(L1)

        except Exception as e:
            print(f"Error processing {hisse}: {e}")

    # Show stocks with confirmed buy signals
    print("Kesinleşmiş Giriş Sinyali Olan Hisseler:")
    print(df_signals)

if __name__ == "__main__":
    main()



  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-bconzhyr
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-bconzhyr
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Maestro Most Tarama // Period Girişi seçenekleri:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Zaman dilimini girişi (1-9) arası rakam : 7
['ADEL', 540.5, 'True']
['AFYON', 15.82, 'True']
['AKCNS', 158.0, 'True']
['ALCTL', 126.5, 'True']
['ALKLC', 29.5, 'True']
['ALTIN', 28.49, 'True']
['ANGEN', 15.13, 'True']
['ARSAN', 22.360001, 'True']
['ARTMS', 55.75, 'True']
['AVPGY', 44.259998, 'True']
['BANVT', 355.0, 'True']
['BIOEN', 19.9, 'True']
['BJKAS', 7.43, 'True']
['BMSTL', 51.049999, 'True']
['BNTAS', 12.15, 'True']
['BORLS', 39.439999, 'True']
['BSOKE', 67.949997, 'True']
['BTCIM', 233.0, 'True']
['CEOEM', 47.66, 'True']
['CIMSA', 35.860001, 'True']
['CMENT', 423.0, 'True']
['CRFSA', 114.4, 'True']
['CVKMD', 542.0, 'True']
['DAGI', 9.91, 'True']
['DAPGM', 5.55, 'True']
['DCTTR', 24.84, 'True']
['DERHL', 10.88, 'True']
['DGGYO', 30.4, 'True']
['DITAS', 22.76, 'True']
['DOCO', 5732.5, 'True']
['DYOBY', 21.559999, 'True']
['EFORC', 2